In [ ]:
import pandas as pd
import spacy

In [ ]:
dados_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
28222,Buscar salvadores que nos defendam com muros é...,"Aludindo ao novo presidente dos EUA, Donald Tr...",2017-01-21,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/01/185...
59603,Edu Dracena entra em acordo com a diretoria e ...,"O zagueiro Edu Dracena, 33, não é mais jogador...",2015-01-15,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/01/1...
43458,Seleção não lamenta adiamento de clássico cont...,"O preparador físico da seleção brasileira, Fáb...",2015-12-11,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/11/1...
76295,Bomba da 2ª Guerra Mundial provoca retirada de...,Cerca de 54 mil pessoas serão retiradas de sua...,2016-12-25,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/12/184...
18099,"Se os anos 70 foram de David Bowie, os 80 fora...","Com as mortes de David Bowie e Prince, a músic...",2016-04-22,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/04...


In [ ]:
#Rodas apenas uma vez
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 89.9MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=e6686ba775d01478f4e0e215883920b63ad81fd819ef08daf6ceeaba8f527e86
  Stored in directory: /tmp/pip-ephem-wheel-cache-tsi63j7g/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [ ]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [ ]:
type(doc[2])

spacy.tokens.token.Token

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [ ]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

In [ ]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)

3.3800423622131346


In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen a o turno frança revés siglas ...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [ ]:
w2v_modelo

In [ ]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
86113


In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2020-05-28 13:18:49,960 : - collecting all words and their counts
2020-05-28 13:18:49,962 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-28 13:18:49,978 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2020-05-28 13:18:49,995 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2020-05-28 13:18:50,011 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2020-05-28 13:18:50,029 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2020-05-28 13:18:50,042 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2020-05-28 13:18:50,059 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2020-05-28 13:18:50,074 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2020-05-28 13:18:50,087 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899